## Import the libraries and modules

Please note that we have written a module for Options Pricing. This module is available in the same folder as this notebook. We will import this module and use it for our calculations. The module contains classes to perform the calculations. We will use the classes to create objects and perform the calculations.





In [3]:
import numpy as np
from binomial_pricing import Option, OptionStyle, OptionRight

## Choosing Number of Steps

We perform a simulation to choose the number of steps to take for pricing the option. We define the class to perform the simulation and then graph it

In [4]:
import plotly.graph_objects as go
from typing import Generator, Iterable

class NumberOfStepsSimulator:
    def __init__(self, option,iterator: Iterable[int]):
        self.initial_option = option
        self.iterator = iterator
        self.simulated_prices = self.simulate_number_of_steps()
            
    
    def simulate_number_of_steps (self) -> Generator[float, None, None]:
        for steps in self.iterator:            
            option = Option(style=self.initial_option.style, right=self.initial_option.right, spot_price=self.initial_option.spot_price, strike_price=self.initial_option.strike_price, time_to_maturity=self.initial_option.time_to_maturity, risk_free_rate=self.initial_option.risk_free_rate, number_of_steps=steps, upside=self.initial_option.upside, downside=self.initial_option.downside, sigma=self.initial_option.sigma)    
            yield option.price()
    

    def graph(self):
        self.steps = [i for i in self.iterator]
        self.prices = [price for price in self.simulated_prices]
        graph = go.Scatter(x=self.steps, y=self.prices, mode='lines+markers')
        return graph

We simulate from 1 to 30 steps. We also instantiate the Option class with the parameters. Number of steps is anchored to 3 just to instantiate. We will decide the final number of steps through the simulation. 

In [5]:
european_call = Option(style=OptionStyle.EUROPEAN, right=OptionRight.CALL, spot_price=100, strike_price=100, time_to_maturity=3, risk_free_rate=0.05, number_of_steps=3, sigma=0.2)
european_put = Option(style=OptionStyle.EUROPEAN, right=OptionRight.PUT, spot_price=100, strike_price=100, time_to_maturity=3, risk_free_rate=0.05, number_of_steps=3, sigma=0.2)
american_call = Option(style=OptionStyle.AMERICAN, right=OptionRight.CALL, spot_price=100, strike_price=100, time_to_maturity=3, risk_free_rate=0.05, number_of_steps=3, sigma=0.2)
american_put = Option(style=OptionStyle.AMERICAN, right=OptionRight.PUT, spot_price=100, strike_price=100, time_to_maturity=3, risk_free_rate=0.05, number_of_steps=3, sigma=0.2)

In [6]:
from plotly.subplots import make_subplots
simulation_plots = make_subplots(rows=2, cols=2, subplot_titles=('European Call','European Put','American Call','American Put'))
simulation_plots.update_layout(height=600, width=1200, title_text="Convergence of Option Prices")
simulation_plots.add_trace(NumberOfStepsSimulator(european_call, range(1,30,1)).graph(), row = 1, col = 1)
simulation_plots.add_trace(NumberOfStepsSimulator(european_put, range(1,30,1)).graph(),row = 1, col = 2)
simulation_plots.add_trace(NumberOfStepsSimulator(american_call, range(1,30,1)).graph(),row = 2, col = 1)
simulation_plots.add_trace(NumberOfStepsSimulator(american_put, range(1,30,1)).graph(),row = 2, col = 2)
simulation_plots.show()


We find that the call price starts oscillating around $21 at arounf 10 steps. We choose number of steps as 3 to also ensure nice divisibility by 3.

In [7]:
european_call = Option(style=OptionStyle.EUROPEAN, right=OptionRight.CALL, spot_price=100, strike_price=100, time_to_maturity=3, risk_free_rate=0.05, number_of_steps=3, sigma=0.2)
european_put = Option(style=OptionStyle.EUROPEAN, right=OptionRight.PUT, spot_price=100, strike_price=100, time_to_maturity=3, risk_free_rate=0.05, number_of_steps=3, sigma=0.2)
american_call = Option(style=OptionStyle.AMERICAN, right=OptionRight.CALL, spot_price=100, strike_price=100, time_to_maturity=3, risk_free_rate=0.05, number_of_steps=3, sigma=0.2)
american_put = Option(style=OptionStyle.AMERICAN, right=OptionRight.PUT, spot_price=100, strike_price=100, time_to_maturity=3, risk_free_rate=0.05, number_of_steps=3, sigma=0.2)

## Calculating Greek Delta

In [13]:
from pandas import DataFrame
european_delta = [round(option.delta_evolution[0,0],2) for option in [european_call, european_put]]
DataFrame(european_delta, index=['European Call', 'European Put'], columns=['Delta']).to_latex()

    

C:\Users\kumar\AppData\Local\Temp\ipykernel_19600\1889638869.py:3: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



'\\begin{tabular}{lr}\n\\toprule\n{} &  Delta \\\\\n\\midrule\nEuropean Call &   0.69 \\\\\nEuropean Put  &  -0.38 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [169]:
import numpy as np
def verify_put_call_parity (call_option, put_option):
    message = "Exponentiating to the power of the risk free rate to discount the strike price results in a rounding off error due to floating point precision. This is why the tolerance is set to 1."
    lhs = call_option.price() + np.exp(-call_option.risk_free_rate * call_option.number_of_steps)*call_option.strike_price
    rhs = put_option.price() + put_option.spot_price
    verify_put_call_parity = np.isclose(lhs, rhs, rtol=1)    
    print(f"lhs: C + Ke^-rt = {lhs:.2f}\nrhs: P + S0 = {rhs:.2f}")
    print(f"Round off: {np.abs(lhs-rhs):.2f}")
    print(f"\n{message}")
    return verify_put_call_parity

Verification of put-call parity for European options

In [170]:
verify_put_call_parity(european_call, european_put)

lhs: C + Ke^-rt = 112.84
rhs: P + S0 = 113.17
Round off: 0.33

Exponentiating to the power of the risk free rate to discount the strike price results in a rounding off error due to floating point precision. This is why the tolerance is set to 1.


True

Verification of put-call parity for American options

In [171]:
verify_put_call_parity(american_call, american_put)


lhs: C + Ke^-rt = 112.84
rhs: P + S0 = 113.17
Round off: 0.33

Exponentiating to the power of the risk free rate to discount the strike price results in a rounding off error due to floating point precision. This is why the tolerance is set to 1.


True

In [11]:
import trinomial_pricing
reload (trinomial_pricing)
from trinomial_pricing import TrinomialCall, TrinomialPut, TrinomialAmerican_C, TrinomialAmerican_P

In [12]:
A_tri_Call_price_array = []
for K in [0.9*100,0.95*100,100,1.05*100,1.1*100]:
    A_tri_option_price_call = TrinomialAmerican_C(100.0, 0.05, 0.3, 2.0, K)
    A_tri_Call_price_array.append(A_tri_option_price_call)
    print("Strike Price $ {:.2f}, Moneyness {:.0f}% -> American Call Option Price is $ {:.2f}".format(K, K/100*100, A_tri_option_price_call.price(800)))

Strike Price $ 90.00, Moneyness 90% -> American Call Option Price is $ 26.24
Strike Price $ 95.00, Moneyness 95% -> American Call Option Price is $ 23.61
Strike Price $ 100.00, Moneyness 100% -> American Call Option Price is $ 21.19
Strike Price $ 105.00, Moneyness 105% -> American Call Option Price is $ 19.00
Strike Price $ 110.00, Moneyness 110% -> American Call Option Price is $ 17.00


In [13]:
A_tri_put_price_array = []
for K in [0.9*100, 0.95*100, 100, 1.05*100, 1.1*100]:
    A_tri_option_price_put = TrinomialAmerican_P(100.0, 0.05, 0.3, 2.0, K)
    A_tri_put_price_array.append(A_tri_option_price_put)
    print("Strike Price $ {:.2f}, Moneyness {:.0f}% -> American Put Option Price is $ {:.2f}".format(K, K/100*100, A_tri_option_price_put.price(800)))

Strike Price $ 90.00, Moneyness 90% -> American Put Option Price is $ 8.34
Strike Price $ 95.00, Moneyness 95% -> American Put Option Price is $ 10.46
Strike Price $ 100.00, Moneyness 100% -> American Put Option Price is $ 12.84
Strike Price $ 105.00, Moneyness 105% -> American Put Option Price is $ 15.50
Strike Price $ 110.00, Moneyness 110% -> American Put Option Price is $ 18.41


In [14]:
E_tri_put_price_array = []
for K in [0.9*100, 0.95*100, 100, 1.05*100, 1.1*100]:
    A_tri_option_price_put = TrinomialCall(100.0, 0.05, 0.3, 2.0, K)
    A_tri_put_price_array.append(A_tri_option_price_put)
    print("Strike Price $ {:.2f}, Moneyness {:.0f}% -> American Put Option Price is $ {:.2f}".format(K, K/100*100, A_tri_option_price_put.price(800)))

Strike Price $ 90.00, Moneyness 90% -> American Put Option Price is $ 26.24
Strike Price $ 95.00, Moneyness 95% -> American Put Option Price is $ 23.61
Strike Price $ 100.00, Moneyness 100% -> American Put Option Price is $ 21.19
Strike Price $ 105.00, Moneyness 105% -> American Put Option Price is $ 19.00
Strike Price $ 110.00, Moneyness 110% -> American Put Option Price is $ 17.00


In [15]:
E_tri_put_price_array = []
for K in [0.9*100, 0.95*100, 100, 1.05*100, 1.1*100]:
    A_tri_option_price_put = TrinomialPut(100.0, 0.05, 0.3, 2.0, K)
    A_tri_put_price_array.append(A_tri_option_price_put)
    print("Strike Price $ {:.2f}, Moneyness {:.0f}% -> American Put Option Price is $ {:.2f}".format(K, K/100*100, A_tri_option_price_put.price(800)))

Strike Price $ 90.00, Moneyness 90% -> American Put Option Price is $ 7.67
Strike Price $ 95.00, Moneyness 95% -> American Put Option Price is $ 9.57
Strike Price $ 100.00, Moneyness 100% -> American Put Option Price is $ 11.67
Strike Price $ 105.00, Moneyness 105% -> American Put Option Price is $ 14.00
Strike Price $ 110.00, Moneyness 110% -> American Put Option Price is $ 16.53


In [9]:
american = TrinomialCall(S0=100, r=0.05, sigma=0.2, mat=3, K=100)

In [10]:
print(american.price(2))

20.097798439928535


In [185]:
tree = TrinomialPut(100.0, 0.0, 0.3, 1.0, 90.0)

In [186]:
tree.price(5)

7.264860920887219

In [ ]:
verify_put_call_parity(american_call, american_put)